# Download reference crystal structures

This Notebook contains the workflow for fetching reference atomic crystal structures from various external sites, and comparing them to the defined crystal prototypes in the iprPy library.

__WARNING:__ The functions called here access and download all matching crystal structures from the external sites.  For convenience, you should ideally limit access to only compositions of interest and/or locally copy the references between computational resources rather than downloading again.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (print_function, division, absolute_import,
                        unicode_literals)
import os
import glob

# http://www.numpy.org/
import numpy as np

# https://pandas.pydata.org/
import pandas as pd

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.lammps as lmp

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.9.alpha


## 1. Load database

In [2]:
database = iprPy.load_database('demo')

## 2. Define elements_set

elements_set lists the elemental systems to fetch reference crystal structures for.

### Option #1: Auto generate

This generates elements_set based on the potentials in the database.

In [3]:
# Specify the potential record style
potential_record_style = 'potential_LAMMPS'

# Specify any potential delimiters
potential_kwargs = {}
potential_kwargs['id'] = '1986--Foiles-S-M--Ag--LAMMPS--ipr1'
#potential_kwargs['pair_style'] = 

In [4]:
elements_set = set()
for potential_record in database.get_records(style=potential_record_style, **potential_kwargs):
    potential = lmp.Potential(potential_record.content)
    elements = potential.elements()
    elements.sort()
    elements_set.add(' '.join(elements))

### Option #2: Manually define

Each entry is a space-delimited string of element tags indicating an element system. For example

- 'Au' will fetch elemental Au crystals
- 'Au Cu' will fetch crystals for AuCu, Au2Cu, AuCu2, ...

In [5]:
#elements_set = set()
#elements_set.add('Fe')
#elements_set.add('Cu')
#elements_set.add('Ni')
#elements_set.add('Cu Ni')

## 3. Fetch Materials Project reference structures

### Specify personal Materials Project API key

In [6]:
mp_api_key_location = 'C:\\users\\lmh1\\Documents\\Materials Project\\API key.txt'

with open(mp_api_key_location) as f:
    mp_api_key = f.read()

### Get structures from Materials Project

In [7]:
for elements in elements_set:
    iprPy.tools.get_mp_structures(elements.split(), api_key=mp_api_key)

## 3. Fetch OQMD reference structures

### Get structures from OQMD

In [8]:
for elements in elements_set:
    try:
        iprPy.tools.get_oqmd_structures(elements.split())
    except:
        print('Failed:', elements)

## 4. Add reference structures to database

In [10]:
database.build_refs(include='reference_crystal')